In [5]:
# 1-second clips  ➜  1.pt, 2.pt, ...
import os, torch, torchaudio
from glob import glob
import random

In [8]:
import os
import torchaudio
import torchaudio.transforms as T
import torch

target_sr = 16000
max_duration = 5
max_length = target_sr * max_duration

target_dir = "./vehicle"
wav_tensor_list = []

# 리샘플러 미리 준비 (동일한 orig_freq일 때만 적용)
resamplers = {}  # orig_sr -> resampler

for root, _, files in os.walk(target_dir):
    for fname in files:
        if fname.lower().endswith(".wav"):
            path = os.path.join(root, fname)
            try:
                waveform, sr = torchaudio.load(path)

                # 모노로 변환
                if waveform.shape[0] > 1:
                    waveform = waveform.mean(dim=0, keepdim=True)

                # 리샘플링 필요할 때만
                if sr != target_sr:
                    if sr not in resamplers:
                        resamplers[sr] = T.Resample(orig_freq=sr, new_freq=target_sr)
                    waveform = resamplers[sr](waveform)

                num_samples = waveform.shape[1]
                if num_samples < max_length:
                    continue  # 5초 미만 제거
                elif num_samples > max_length:
                    waveform = waveform[:, :max_length]

                wav_tensor_list.append(waveform.squeeze(0))  # (1, N) → (N,)
            except Exception as e:
                print(f"오류: {path}, {e}")

print(f"총 {len(wav_tensor_list)}개의 5초짜리 wav 텐서를 저장했습니다.")


총 9419개의 5초짜리 wav 텐서를 저장했습니다.


In [9]:

# 셔플
random.seed(42)  # 재현 가능성 위한 고정 시드
random.shuffle(wav_tensor_list)

# 나누기
n_total = len(wav_tensor_list)
n_train = int(n_total * 0.9)

train_list = wav_tensor_list[:n_train]
valid_list = wav_tensor_list[n_train:]

print(f"학습 데이터 개수: {len(train_list)}")
print(f"검증 데이터 개수: {len(valid_list)}")

학습 데이터 개수: 8477
검증 데이터 개수: 942


In [10]:
torch.save(train_list,"train.pt")
torch.save(valid_list,"eval.pt")